In [1]:
## ---------------------------------------------------------------------
## set up configs for huggingface hub and OS paths on HPC cluster -- make sure config.ini is correct
## ---------------------------------------------------------------------
import configparser

def scratch_path():
    config = configparser.ConfigParser()
    config.read("config.ini")
    return "/scratch/" + config["user"]["username"]

import os
if os.path.isdir(scratch_path()):
    os.environ['TRANSFORMERS_CACHE'] = scratch_path() + '/.cache/huggingface'
    os.environ['HF_DATASETS_CACHE'] = scratch_path() + '/.cache/huggingface/datasets'
print(os.getenv('TRANSFORMERS_CACHE'))
print(os.getenv('HF_DATASETS_CACHE'))

## ---------------------------------------------------------------------
## Load libraries
## ---------------------------------------------------------------------

import numpy as np
import pandas as pd

import torch
import transformers
from transformers import AutoTokenizer, AutoModel, LlamaForCausalLM, LlamaTokenizer, AutoModelForCausalLM

import torch.nn.functional as F

from baukit import Trace

from steering import *
## ---------------------------------------------------------------------
## Ensure GPU is available -- device should == 'cuda'
## ---------------------------------------------------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device = ", device)

/scratch/dmpowell/.cache/huggingface
/scratch/dmpowell/.cache/huggingface/datasets


/home/dmpowell/.conda/envs/py3.11transformers4.44/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


device =  cuda


In [98]:
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
# MODEL_NAME = "meta-llama/Llama-3.1-8B"

wmodel = SteeringModel(
    AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,  # Replace this with the 70B variant if available
        torch_dtype=torch.float16,
        device_map=device  # Automatically distributes the model across available GPUs
    ),
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, device = 'cuda', use_fast = False)
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Multiple choice

Here is a basic implementation of multiple choice answering using "cloze" probabilities. This should roughly work with both raw and instruction-tuned models.

In [17]:
import re

def answer_choice_list(choices):
    options = re.split(r'\s*\(\w\)\s*', choices)
    return( [option.strip() for option in options if option] )


def format_question(question):
    return f"Q: {question}\nA:"


def format_statement(question):
    return f"Please rate your agreement with the following statement. Statement: {question}\nResponse:"


def format_chat_question(question):
    return f"Please rate your agreement with the following statement. Statement: {question}"


def format_chat(question):

    chat = [
        {"role": "user", "content": format_chat_question(row['original_statement'])},
        {"role": "system", "content": "My Response:"}
    ]

    tokens = wmodel.tok.apply_chat_template(chat, tokenize=True, continue_final_message=True)[:-1]

    return(wmodel.tok.decode(tokens))


def mc_choice_probs(model, question, choices, pad = True):
    prompt = question
    if pad:
        choices = [" " + c for c in choices] # pad all the 
    
    prompts = [prompt for c in choices]
    
    logits = torch.tensor([model.completion_logprob(x[0], x[1]) for x in zip(prompts, choices)])
    
    return(F.log_softmax(logits, -1).exp())


def choice_score(choice_probs):
    # calculate score on -1 to 1 scale
    choice_score01 = choice_probs @ torch.arange(len(choice_probs), dtype = choice_probs.dtype)/(len(choice_probs)-1)
    return (choice_score01.item() - .5)*2

For any agree/disagree etc. style scales, we can take the choice probabilities and compute a "score". I noticed the model seems to have a really strong "agree" bias when we have a pure "agree" option. Will need to look into this, probably some literature on it.

In [4]:
choice_probs = mc_choice_probs(wmodel, format_question('Slavery benefitted the slaves, many of whom learned valuable skills.'), ['Strongly disagree', 'Somewhat disagree', "Neither agree nor disagree", 'Somewhat agree', 'Strongly agree'])
choice_score(choice_probs), choice_probs

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


(-0.8667596280574799, tensor([0.8655, 0.0718, 0.0178, 0.0207, 0.0242]))

## Steering

Applying a steering vector shifts generations ...

In [74]:
def get_mean_steering_vector(ziplist, model):
    return torch.concat([model.get_steering_vector(s1, s2) for s1, s2 in ziplist]).mean(0).unsqueeze(0)


def act_add(steering_vec):
    def act_add(output):
        z = torch.ones_like(output[0])
        z = z * steering_vec
        out = output[0] + z
        return (out, output[1])
    return act_add


steering_vec = get_mean_steering_vector([["Love", "Hate"], ["Beautiful", "Disgusting"], ["Loving", "Hateful"]], wmodel)

test_sentence = "In my opinion, dogs"

print(wmodel.generate(test_sentence, max_new_tokens = 20))
print("-"*20)
with Trace(wmodel.get_module(), edit_output = act_add(steering_vec)):
    print(wmodel.generate(test_sentence, max_new_tokens = 20))
print("-"*20)
with Trace(wmodel.get_module(), edit_output = act_add(-1.5*steering_vec)):
    print(wmodel.generate(test_sentence, max_new_tokens = 20))

['In my opinion, dogs are one of the most loving and loyal animals on this planet. They bring so much joy and companions']
--------------------
['In my opinion, dogs are the best companions you can have. They are loyal, gentle, and loving. They are also']
--------------------
['In my opinion, dogs are the worst. They are everywhere, everywhere, everywhere. On the streets, on public transportation,']


### "survey" responses and attitude scores

Can apply the same thing to our multiple choice decisions and calculated scores. Adding a relevant steering vector affects the responses.

In [9]:
steering_vec = wmodel.get_steering_vector("Slavery is a proud part of American history.", "Slavery is the most shameful part of America's past.")

# steering_vec = get_mean_steering_vector([["Some races are inferior to others.", "No race is inferior to others."], ["White people are superior to black people.", "White and black people are equally good."]], wmodel)

with Trace(wmodel.get_module(), edit_output = act_add(steering_vec)):
    # print(wmodel.generate(test_sentence, max_new_tokens = 20))
    choice_probs = mc_choice_probs(wmodel, format_question('Slavery benefitted the slaves who learned valuable skills.'), ['Strongly disagree',  'Somewhat disagree', "Neither agree nor disagree", 'Somewhat agree', 'Strongly agree'])
    print(choice_score(choice_probs), choice_probs) 

with Trace(wmodel.get_module(), edit_output = act_add(-steering_vec)):
    # print(wmodel.generate(test_sentence, max_new_tokens = 20))
    choice_probs = mc_choice_probs(wmodel, format_question('Slavery benefitted the slaves who learned valuable skills.'), ['Strongly disagree',  'Somewhat disagree', "Neither agree nor disagree", 'Somewhat agree', 'Strongly agree'])
    print(choice_score(choice_probs), choice_probs) 

-0.6855780482292175 tensor([0.8168, 0.0296, 0.0062, 0.0029, 0.1445])
-0.9813397862017155 tensor([9.8887e-01, 1.2226e-03, 4.3364e-04, 2.6493e-03, 6.8207e-03])


## Applying to survey ideology scales

First, to generate the model's answers.

In [99]:
scales = pd.read_csv("data/scales.tsv", sep="\t")

In [103]:
## Getting model responses
resps = []
resp_probs = []

for idx, row in scales.iterrows():
    choices = re.split(';\W', row['response'])
    choices = [c.strip() for c in choices]
    choice_probs = mc_choice_probs(wmodel, format_chat(row['original_statement']), choices) # format_chat for instruct model

    resp_probs.append(choice_probs.detach().numpy())
    resps.append(choice_score(choice_probs) if row['direction']=='high' else -choice_score(choice_probs))
    
scales["response_probs"] = resp_probs
scales["model_score"] = resps

In [105]:
# answer_choice_list('Strongly disagree, Somewhat disagree, Neither agree nor disagree, Somewhat agree, Strongly agree')

scales.groupby(['scale', 'sub_scale']).agg(avg_score = ('model_score', 'mean'))

avg_score
scale sub_scale                                
CSES  Importance to Identity           0.003708
      Membership self-esteem.          0.011190
      Private collective self-esteem   0.016076
      Public collective self-esteem    0.014440
IPVAS Control                         -0.633278
      Threat                          -0.333069
      Violence                        -0.999981
LWAI  Anticonventionalism              0.000000
      Antihierarchical Aggression      0.000000
      Top-Down Censorship              0.000000
MFQ   Authority                       -0.017626
      Fairness                         0.081313
      Harm                             0.052985
      Ingroup                          0.032067
      Purity                           0.018520
      not scored                      -0.849965
SDO-7 trait antiegalitarianism        -0.972336
      trait dominance                 -0.682363

## quick steering test

### On the instruct model

- model steering with a subset of SDO items does affect SDO -- so that's promising! it does immediately pass a really basic/naive test.
- AND, it also spills over to substantailly affect IPVAS, suggesting some generalization.


### on the non-instruct model

- seemingly it is not affected, which is strange. Also scores very strangely in raw tests

In [106]:
sdo = scales.loc[lambda x: ((x.scale == "SDO-7") & (x.direction == 'high'))]
sdo_zipped = zip(sdo.original_statement.to_list(), sdo.contrastive_statement.to_list())
sdo_vec = get_mean_steering_vector(sdo_zipped, wmodel)

## Getting model responses

resps = []
resp_probs = []

for idx, row in scales.iterrows():

    with Trace(wmodel.get_module(), edit_output = act_add(2*steering_vec)):
        choices = re.split(';\W', row['response'])
        choices = [c.strip() for c in choices]
        choice_probs = mc_choice_probs(wmodel, format_chat(row['original_statement']), choices) # format_chat for instruct model

        resp_probs.append(choice_probs.detach().numpy())
        resps.append(choice_score(choice_probs) if row['direction']=='high' else -choice_score(choice_probs))
    
scales["response_probs"] = resp_probs
scales["model_score"] = resps

scales.groupby(['scale', 'sub_scale']).agg(avg_score = ('model_score', 'mean'))


avg_score
scale sub_scale                                
CSES  Importance to Identity           0.009327
      Membership self-esteem.          0.004570
      Private collective self-esteem  -0.011315
      Public collective self-esteem    0.032840
IPVAS Control                         -0.508341
      Threat                           0.052839
      Violence                        -0.741021
LWAI  Anticonventionalism              0.000000
      Antihierarchical Aggression      0.000000
      Top-Down Censorship              0.000000
MFQ   Authority                       -0.002137
      Fairness                         0.001122
      Harm                             0.001027
      Ingroup                         -0.001178
      Purity                          -0.001007
      not scored                      -0.608582
SDO-7 trait antiegalitarianism        -0.017029
      trait dominance                 -0.027916